# Text2SQL with watsonx.ai & DB2 using Python SDK

### Install Discovery SDK Import the Dependencies

In [1]:
#!pip ibm-watson-machine-learning
#!pip install ibm-db

In [2]:
import sys
import json
import requests
import ast, os
import pandas as pd
import time
import ibm_db, ibm_db_dbi as dbi

from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [3]:
WX_API_KEY = "IAM_APIKEY"
WX_PROJECT_ID = "WATSONX.AI_PROJECTID"
WX_URL= "https://us-south.ml.cloud.ibm.com"

DB2_HOST = "DB2 HOST"
DB2_PORT = "32731"
DB2_USERNAME = "DB2 USERNAME"
DB2_PASSWORD = "DB2 PASSWORD"
DB2_SCHEMA= "DB2 SCHEMA NAME"
DB2_TABLE= "DB2 TABLE NAME"

creds = {
    "url": WX_URL,
    "apikey": WX_API_KEY 
}

### Function for DB2

In [4]:
#=============================DB2 Function==============================
def db2_init(DB2_HOST=DB2_HOST, DB2_PORT=DB2_PORT, DB2_USERNAME=DB2_USERNAME, DB2_PASSWORD=DB2_PASSWORD):
    db2_dsn = 'DATABASE={};HOSTNAME={};PORT={};PROTOCOL=TCPIP;UID={uid};PWD={pwd};SECURITY=SSL'.format(
        'bludb',
        DB2_HOST,   
        DB2_PORT,         
        uid=DB2_USERNAME,     
        pwd=DB2_PASSWORD     
    )

    db2_connection = dbi.connect(db2_dsn)
    return db2_connection

db2_connection = db2_init()

def query_db2_df(query, db2_connection = db2_connection):    
    answer_df = pd.read_sql_query(query, con=db2_connection)
    return answer_df

#=============================Get from main table==============================
table_name = f"{DB2_SCHEMA}.{DB2_TABLE}"
query_init = f"SELECT * FROM {table_name}"
answer_df = query_db2_df(query_init)
header = list(answer_df.columns)

/tmp/wsuser/ipykernel_185/273475841.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_df = pd.read_sql_query(query, con=db2_connection)


### Function for watsonx.ai

In [5]:
def send_to_watsonxai(prompt, creds=creds, project_id=WX_PROJECT_ID,
                    model_name='meta-llama/llama-3-70b-instruct', #'mistralai/mixtral-8x7b-instruct-v01',', #'meta-llama/llama-2-13b-chat', #
                    decoding_method="greedy",
                    max_new_tokens=300,
                    min_new_tokens=1,
                    temperature=0,
                    repetition_penalty=1.0,
                    stop_sequences=[],
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompt)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
        GenParams.STOP_SEQUENCES: stop_sequences
    }

    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)
    
    
    output = model.generate_text(prompt)
    return output


def send_to_watsonxai_stream(prompt, creds=creds, project_id=WX_PROJECT_ID,
                    model_name='meta-llama/llama-3-70b-instruct', #'mistralai/mixtral-8x7b-instruct-v01',', #'meta-llama/llama-2-13b-chat', #
                    decoding_method="greedy",
                    max_new_tokens=300,
                    min_new_tokens=1,
                    temperature=0,
                    repetition_penalty=1.0,
                    stop_sequences=[],
                    ):
    '''
   helper function for sending prompts and params to Watsonx.ai
    
    Args:  
        prompts:list list of text prompts
        decoding:str Watsonx.ai parameter "sample" or "greedy"
        max_new_tok:int Watsonx.ai parameter for max new tokens/response returned
        temperature:float Watsonx.ai parameter for temperature (range 0>2)
        repetition_penalty:float Watsonx.ai parameter for repetition penalty (range 1.0 to 2.0)

    Returns: None
        prints response
    '''

    assert not any(map(lambda prompt: len(prompt) < 1, prompt)), "make sure none of the prompts in the inputs prompts are empty"

    # Instantiate parameters for text generation
    model_params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.RANDOM_SEED: 42,
        GenParams.TEMPERATURE: temperature,
        GenParams.REPETITION_PENALTY: repetition_penalty,
        GenParams.STOP_SEQUENCES: stop_sequences
    }

    # Instantiate a model proxy object to send your requests
    model = Model(
        model_id=model_name,
        params=model_params,
        credentials=creds,
        project_id=project_id)
    
    output = model.generate_text_stream(prompt)
    # output = model.generate_text(prompt) # This is for not streaming
    for chunk in output:
        yield chunk

### Function for Text to SQL

In [6]:
def question_to_sql(user_question, table_name=table_name,header=header):
    prompt_query = f"""
        Anda adalah asisten yang bertugas untuk mengkonversi pertanyaan ke dalam bentuk SQL. Buatlah kueri SQL berdasarkan dari 2 tabel informasi beserta kolom pada "table_schema" untuk menjawab pertanyaan dalam bentuk format JSON.
        table_schema: 
        table: {table_name}
        columns: {header}
        
        Informasi penting:
        - Kolom yang difilter tampilkan juga pada list kolom
        - Setiap menampilkan kolom Salesman atau customer atau invoice atau supervisor harus menampilkan juga kolom Location
        - Setiap menampilkan kolom Invoice sertakan juga informasi customer dan lokasi
        - Tampilkan informasi sesuai yang ditanyakan, tidak perlu melakukan relasi ke table yang tidak ditanyakan
        - Istilah Cabang berarti Plant dan Stockpoint berarti Salespoint. Atau sebaliknya
        - Pertanyaan yang terkait nama lokasi, nama salespoint, nama plant, nama cabang gunakan filter WHERE LIKE '%'. Jika nilai dari nama mengandung spasi, maka berikan filter dengan memberikan spasi dan tanpa spasi
        - Pada table ProductMaster tidak perlu lakukan link join kolom LocationID
        - Kolom yang difilter tampilkan juga pada list kolom
        - Setiap menampilkan kolom Salesman atau customer atau invoice atau supervisor harus menampilkan juga kolom Location
        - Istilah Cabang berarti Plant dan Sockpoint berarti Salespoint. Atau sebaliknya
        - Pertanyaan yang terkait nama lokasi, nama salespoint, nama plant, nama cabang gunakan filter WHERE LIKE '%'. Jika nilai dari nama mengandung spasi, maka berikan filter dengan memberikan spasi dan tanpa spasi
        Contoh pertanyaan dan kueri SQL:
        Pertanyaan: Tampilkan top 5 salesman dengan total penjualan tertinggi
        Jawaban: {{"query": "SELECT SALESMANNAME, SALESMANCITY, SUM(NETSALESAMOUNT) AS TotalSalesAmount FROM {table_name} GROUP BY SALESMANNAME, SALESMANCITY ORDER BY TotalSalesAmount DESC LIMIT 5;"}}
        Pertanyaan: Tampilkan customer yang berada di kota Bogor
        Jawaban: {{"query": "SELECT DISTINCT CUSTOMERNAME, CUSTOMERPHONE, CUSTOMERGROUP, CUSTOMERADDRESS, CUSTOMERCOUNTRY, CUSTOMERTYPE FROM {table_name} WHERE UPPER(CUSTOMERCITY) = UPPER('Bogor');"}}
        Pertanyaan: Perusahaan ABC ada di kota apa saja?
        Jawaban: {{"query": "SELECT DISTINCT COMPANYNAME, CITY FROM {table_name} WHERE COMPANYNAME = 'PT. ABC';"}}
        Pertanyaan: Tampilkan top 5 produk dengan total penjualan terbanyak
        Jawaban: {{"query": "SELECT PRODUCTID, SHORTNAME, SUM(NETSALESQUANTITY) AS TotalSalesQuantity FROM {table_name} GROUP BY PRODUCTID, SHORTNAME ORDER BY TotalSalesQuantity DESC LIMIT 5;"}}
        
        Pertanyaan:"Kapan paling lambat customer toko ismail bayar invoicenya?"
        Jawaban: {{"query": "SELECT MAX(DUEDATE) AS LatestDueDate FROM {table_name} WHERE LOWER(CUSTOMERNAME) LIKE LOWER('%TOKO ismail%');"}}
        Pertanyaan: Berapa yang harus dibayar oleh customer dua sultan dan berapa yang tersisa?
        Jawaban: {{"query": "SELECT SUM(LASTAMOUNT) AS TotalAmountToPay, SUM(LASTAMOUNT - PAYMENTAMOUNT) AS RemainingAmount FROM {table_name} WHERE LOWER(CUSTOMERNAME) LIKE LOWER('%dua sultan%');"}}
        
        Pertanyaan: Siapa saja yang disupervisi oleh Jupri?
        Jawaban: {{"query": "SELECT DISTINCT SALESMANNAME FROM {table_name} WHERE LOWER(SUPERVISORNAME) LIKE LOWER('%Jupri%');"}}
        
        Pertanyaan: Tampilkan salesman di Surabaya?
        Jawaban: {{"query": "SELECT DISTINCT SALESMANNAME, SALESMANCITY, SALESMANPHONE, SALESMANSTATUS FROM {table_name} WHERE LOWER(SALESMANCITY) = LOWER('Surabaya');"}}
                
        Pertanyaan: {user_question}
        Jawaban:
        """

    try:
        query = send_to_watsonxai(prompt_query, model_name='meta-llama/llama-3-8b-instruct', min_new_tokens=2,
                                           max_new_tokens=300, stop_sequences=["\n\n", "}"])
        print(f"{query}\n")
        query = query.strip().split("\n")[0]
        query = ast.literal_eval(query)['query']
        return query
            
    except:
        return "error: can't generate SQL queries properly"

In [7]:
user_question = "Tampilkan top 5 salesman dengan total penjualan tertinggi"
query = question_to_sql(user_question)

 {"query": "SELECT SALESMANNAME, SALESMANCITY, SUM(NETSALESAMOUNT) AS TotalSalesAmount FROM SWS48026.IBMMAIN GROUP BY SALESMANNAME, SALESMANCITY ORDER BY TotalSalesAmount DESC LIMIT 5;"}



### Get data from DB2 using the Generated Query

In [8]:
query_result = query_db2_df(query)
data = query_result.to_dict(orient="records")
print(f"{data}\n")

[{'SALESMANNAME': 'ATANG SUPRIADI', 'SALESMANCITY': 'SEMARANG', 'TOTALSALESAMOUNT': 338256450000}, {'SALESMANNAME': 'FAHMY DEVY RAMADHAN', 'SALESMANCITY': 'JAKARTA', 'TOTALSALESAMOUNT': 309005340000}, {'SALESMANNAME': 'HERI NURYADI', 'SALESMANCITY': 'SEMARANG', 'TOTALSALESAMOUNT': 286507160000}, {'SALESMANNAME': 'ACMAD IGOR', 'SALESMANCITY': 'SURABAYA', 'TOTALSALESAMOUNT': 272829700000}, {'SALESMANNAME': 'AGUNG TAUFIKKUROHMAN', 'SALESMANCITY': 'TASIKMALAYA', 'TOTALSALESAMOUNT': 267050340000}]



/tmp/wsuser/ipykernel_185/273475841.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  answer_df = pd.read_sql_query(query, con=db2_connection)


In [9]:
def query_wxai(user_question, data, streaming=False):
    print(user_question)
    start_time = time.time()

    if data ==[]:
        print("NO DATA")
        return {'answer': "Maaf, informasi yang anda butuhkan tidak tersedia di database, silahkan coba dengan pertanyaan lain! Terima kasih."}
    else:
        prompt = f"""
            Anda adalah customer service dari perusahaan IBM yang bertugas untuk menjawab semua pertanyaan terkait data Sales, data Piutang, dan data Pembayaran.
            Tugas Anda adalah untuk memberikan jawaban yang baik, ramah, dan menarik berdasarkan pertanyaan user_question dan hasil query_result yang diberikan.
            user_question: {user_question}
            query_result: {data}
    
            Instruksi analisa:
            1. Jika query_result mengalami error, jawablah dengan sopan bahwa Anda tidak bisa menjawabnya.
            2. Jangan menjawab pertanyaan apapun apabila Anda tidak tahu jawabannya.
            3. Apabila pertanyaan mengandung jumlah besaran angka, formatlah angka tersebut sebagai uang dengan menggunakan desimal dan standar rupiah.
            4. Jika jawaban ditulis dengan huruf kapital semua, ubah menjadi format yang mengikuti kaidah bahasa Indonesia.
            5. Jika tidak bisa menjawab pertanyaan user dengan benar, maka jawablah dengan 'Maaf informasi yang ada tidak cukup untuk menjawab pertanyaan anda'.
            6. Hindari penggunaan baris baru saat menjawab.
            7. Jawablah pertanyaanya menjadi menjadi suatu kalimat yang menjelaskan query_result
            8. Jangan menambahkan informasi apapun selain penjelasan terkait "query_result"
            Jawaban:"""
        
    print("streaming =", streaming)

    if streaming:
        return send_to_watsonxai_stream(prompt, creds=creds, project_id=WX_PROJECT_ID, model_name='meta-llama/llama-3-70b-instruct', min_new_tokens=8,
                                            max_new_tokens=300, stop_sequences=["\n\n", "}"])

    else:
        result = send_to_watsonxai(prompt, creds=creds, project_id=WX_PROJECT_ID, model_name='meta-llama/llama-3-70b-instruct', min_new_tokens=8,
                                            max_new_tokens=300, stop_sequences=["\n\n", "}"])
        eta_wx = time.time() - start_time
        print("eta_wx: ", eta_wx)
        return result, eta_wx

In [10]:
query_wxai(user_question, data)

Tampilkan top 5 salesman dengan total penjualan tertinggi
streaming = False
eta_wx:  8.949172735214233


(' Berikut adalah top 5 salesman dengan total penjualan tertinggi: Atang Supriadi dari Semarang dengan total penjualan sebesar Rp 338.256.450.000, Fahmy Devy Ramadhan dari Jakarta dengan total penjualan sebesar Rp 309.005.340.000, Heri Nuryadi dari Semarang dengan total penjualan sebesar Rp 286.507.160.000, Acmad Igor dari Surabaya dengan total penjualan sebesar Rp 272.829.700.000, dan Agung Taufikkuroman dari Tasikmalaya dengan total penjualan sebesar Rp 267.050.340.000.',
 8.949172735214233)

In [11]:
for chunk in query_wxai(user_question, data, streaming=True):
    print(chunk, end='')

Tampilkan top 5 salesman dengan total penjualan tertinggi
streaming = True
 Berikut adalah top 5 salesman dengan total penjualan tertinggi: Atang Supriadi dari Semarang dengan total penjualan sebesar Rp 338.256.450.000, Fahmy Devy Ramadhan dari Jakarta dengan total penjualan sebesar Rp 309.005.340.000, Heri Nuryadi dari Semarang dengan total penjualan sebesar Rp 286.507.160.000, Acmad Igor dari Surabaya dengan total penjualan sebesar Rp 272.829.700.000, dan Agung Taufikkuroman dari Tasikmalaya dengan total penjualan sebesar Rp 267.050.340.000.